In [ ]:
import os
from os.path import basename, dirname, join, exists
import sys
import glob 
import time

import numpy as np
import numpy.ma as ma
import scipy.stats as stats
import pandas as pd
from scipy.optimize import minimize
import geopandas as gpd 

from datetime import datetime
from datetime import timedelta
from time import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as lines

# from pandas.tseries import converter

In [ ]:
doc_dir = os.getcwd()
while basename(doc_dir) != 'Documents':
    doc_dir = dirname(doc_dir)

# git dir
git_dir = join(doc_dir,'GitHub')

# dir of all gwfm data
gwfm_dir = join(dirname(doc_dir),'Box/research_cosumnes/GWFlowModel')

flopy_dir = doc_dir+'/GitHub/flopy'
if flopy_dir not in sys.path:
    sys.path.insert(0, flopy_dir)
import flopy 

import flopy.utils.binaryfile as bf


In [ ]:
# set box directory for output figures and data
box_dir = gwfm_dir+'/Levee_setback/levee_setback_distance_analysis/'

# tprogs_id = '' # original tprogs with conditioning data in output tsim
tprogs_id = '_no_conditioning'

data_dir = box_dir+ tprogs_id+'/data_output/'
fig_dir = box_dir+tprogs_id+'/figures/'

chan_dir = box_dir+'channel_data/'
gis_dir = chan_dir+'GIS/'

In [ ]:
git_dir = join(doc_dir,'GitHub')
fxn_dir = git_dir+'/python_utilities'
if fxn_dir not in sys.path:
    sys.path.append(fxn_dir)
# sys.path
# import muskingum_recharge as mr

from importlib import reload
# reload(mr)

# Regional Model Connec3D input

In [ ]:
# np.reshape fortran- F = first index is fastest, last is slowest, C = last is fastest, first is slowest
# CONNEC3D assumes Z is fastest then Y then X

In [ ]:

tprogs_id = '_no_conditioning'
mf_tprogs_dir = gwfm_dir+'/UPW_data/tprogs_final' + tprogs_id+'/'
tprogs_files = glob.glob(mf_tprogs_dir+'*')
# tprogs_files


In [ ]:
# z, y, x
nlay, nrow, ncol = (320, 100,230)
arr_dim = (nlay,nrow,ncol)

In [ ]:
ext_dir = 'F:/WRDAPP'
c_dir = 'C:/WRDAPP'

if os.path.exists(ext_dir):
    loadpth = ext_dir 
elif os.path.exists(c_dir):
    loadpth = c_dir 

loadpth = loadpth +'/GWFlowModel/Cosumnes/levee_setback/setback_distance_analysis/'
model_ws = loadpth+'Connec3d' + tprogs_id
os.makedirs(model_ws, exist_ok=True)

In [ ]:
def add_path(fxn_dir):
    if fxn_dir not in sys.path:
        sys.path.append(fxn_dir)
        
add_path(doc_dir+'/GitHub/CosumnesRiverRecharge/tprogs_utilities')
add_path(doc_dir+'/GitHub/CosumnesRiverRecharge/python_utilities')

In [ ]:

import tprogs_cleaning as tc
import tprogs_permeameter as tp

from importlib import reload
# importlib.reload
reload(tc)

In [ ]:
# a basic connectivity run takes about 1 hour (7610 components), good enough in parallel with python
# might be worth trying to compile with ifortran to speed things up
# adding a compiler option seemed to speed things up
# reviewing CCO output showed a bad result (didn't line up with input) reshape order='F' is not needed

the runtime is incredibly long so I"m going to remove unnecessary cells from the calculation 
drop cells above land surface and cells outside of floodplain (3,000m limit) with a buffer for lateral connectivity
only 36 percent of coarse are within the floodplain  
-> decided not to do this so that in the future if someone wants it then it's available on the full scale.


In [ ]:
dem_data = np.loadtxt(gwfm_dir+'/DIS_data/dem_52_9_200m_mean.tsv')

grid_sfr = gpd.read_file(gwfm_dir+'/SFR_data/final_grid_sfr/grid_sfr.shp')
grid_p = gpd.read_file(gwfm_dir+'/DIS_data/grid/grid.shp')

str_setbacks = np.zeros((nrow,ncol))
buf_sfr = grid_sfr.copy()
# give lateral buffer of 600 to account for mean length
buf_sfr.geometry = grid_sfr.buffer(3800)
grid_sfr_buf = gpd.sjoin(grid_p,buf_sfr, how='right', lsuffix = 'grid', rsuffix = 'sfr',predicate='within')
grid_sfr_buf = grid_sfr_buf.drop_duplicates('node_grid')    
str_setbacks[grid_sfr_buf.row_grid.values-1,grid_sfr_buf.column_grid.values-1] = 1


In [ ]:
tprogs_info = [80, -80, 320]

# masked_tprogs = tc.tprogs_cut_elev(tprogs_line, dem_data, tprogs_info)


In [ ]:
import h5py
f = h5py.File(join(chan_dir, 'setback_locs.hdf5'), "r")
local_str_setbacks = f['setbacks']['local'][:]
str_setbacks = f['setbacks']['regional'][:]

f.close()

## Iterate and write .dat files
On 2023-5-25 I tested the impact if we cropped data above the DEM before running connectivity as there might be lateral connecting components above ground that don't actually exist.

In [ ]:
# convert any data above ground to the background (0)
# bottom elevations array of tprogs
bot_elev = np.reshape(np.flip(np.arange(-80,80,0.5)), (320, 1,1))
bot_elev = np.repeat(np.repeat(bot_elev, 100, axis=1), 230, axis=2)

# make any data above the DEM a -1 and crop out
# conn_arr[bot_elev>dem_data] = 0


In [ ]:
model_ws, dat_nam

In [ ]:
# the dat files are slow to write and are better run with multiprocess
# write dat file input
# for t in np.arange(0,100):
for t in [0,9]:
    tprogs_line = np.loadtxt(tprogs_files[t])
    # convert any negatives representing input data to same value
    tprogs_arr = np.abs(np.reshape(tprogs_line, (320, 100,230)))
    conn_arr = np.zeros((320, 100,230))
    # new array where sand, gravel (1,2) are 1-coarse and sandy mud, mud are 0-fine
    conn_arr[(tprogs_arr == 1)|(tprogs_arr == 2)] = 1
    # AJC 2023-5-25, make any data above the DEM (plus 1 m buffer) a 0 and crop out
    conn_arr[bot_elev>dem_data+1] = 0
    # AJC new 2023-4-21, because CCO don't align with distinct units
    # change shape to be X, Y, Z for Connec3D
    conn_arr_t = np.transpose(conn_arr)
    # convert to z,y, x order
    conn_line = np.reshape(conn_arr_t, (320*100*230) ) #, order='F')

    dat_nam = 'r'+str(t).zfill(3)+'.DAT'
    np.savetxt(join(model_ws,  dat_nam), conn_line.astype(int))

In [ ]:
# error checking
# tprogs_arr.shape, conn_arr_t.shape

In [ ]:
# relatively small files so okay to run from notebook
# write par file input
# for t in np.arange(0,1):
# for t in [0,9]:
    dat_nam = 'r'+str(t).zfill(3)+'.DAT'
    connec_id = 1
    connec_pts = 6 # 6, 18, 26 point connectivity (faces, edge, corners)
    # dat file name
    # number of cells in x,y,z
    # x,y,z cell lengths
    # lags, Alisha recommended 30
    out_par = [str(connec_id),str(connec_pts), dat_nam, '230 100 320', '200 200 0.5', '30']
    out_par = '\n'.join(out_par) +'\n'

    with open(model_ws+ '/r'+str(t).zfill(3)+'.PAR', 'w') as f:
        f.write(out_par)
        # output file names
        f.write('r'+str(t).zfill(3)+'.STA\n')
        f.write('r'+str(t).zfill(3)+'.CCO\n')
        f.write('r'+str(t).zfill(3)+'.COF\n')



In [ ]:
# Not used since multiprocess works
# write batch file input
# with open(model_ws+ '/00_run_all_series.bat', 'w') as f:
#     for t in np.arange(0,100):
#         file = 'r'+str(t).zfill(3)+'.PAR'
#         runline = 'echo '+ file +' | Connec3DLarge.exe\n'
#         f.write(runline) 


## Check output

In [ ]:
t = 0
tprogs_line = np.loadtxt(tprogs_files[t])
# convert any negatives representing input data to same value
tprogs_arr = np.abs(np.reshape(tprogs_line, (320, 100,230)))
conn_arr = np.zeros((320, 100,230))
# new array where sand, gravel (1,2) are 1-coarse and sandy mud, mud are 0-fine
conn_arr[(tprogs_arr == 1)|(tprogs_arr == 2)] = 1

In [ ]:
# tprogs_hist = np.histogram(tprogs_arr, bins=[0,1.1,2.1,3.1,4.1])
# tprogs_hist[0]/(320*100*230)

In [ ]:
# Connec3D Connected Components Output file
# cco_in = np.loadtxt(join(model_ws, 'r000.CCO'))
cco_in = np.loadtxt(join(model_ws, 'r000.CCO'))
# reshape to array format
# cco = np.reshape(cco_in, arr_dim)
# account for transposing before input
cco = np.reshape(cco_in, np.flip(arr_dim))
cco = np.transpose(cco)


# Process output
I didn't do this originally but technically the output should be flipped along the y and z. Since the data for connec3d didn't use conditioning data then there isn't really a need to flip since the output should essentially be all random.
- I did not re-run the C3D processing to remaing consistent for the setback distance analysis that there is not flipping which must be consistent when sampling for the conductivity

In [ ]:
box_dir = gwfm_dir+'/Levee_setback/levee_setback_distance_analysis/'

# tprogs_id = '' # original tprogs with conditioning data in output tsim
# tprogs_id = '_no_conditioning'
tprogs_id = '_no_cond_c3d'

data_dir = box_dir+ tprogs_id+'/data_output/'
fig_dir = box_dir+tprogs_id+'/figures/'

In [ ]:
def get_cco_vert(cco, dem_data):
    # identify the connected components, each has a unique number
    z,y,x = np.where(cco>0)
    val = cco[z,y,x]
    # np.transpose((z,y,x,val))
    # create a dataframe of the locations of connected components
    df = pd.DataFrame(np.transpose((z,y,x,val)),columns=['z','y','x','cc'])
    # find ground elevation at each connected point
    df['dem'] = dem_data[df.y.astype(int),df.x.astype(int)]
    # calculate elevation from tprogs layer
    df['elev'] = 80-z*0.5
    # check whether each layer is above land surface
    df['above_gse'] = (df.elev > df.dem) 
    # check wehether each layer is 30 m below land surface (100 ft)
#     df['below_30'] = (df.elev <= df.dem - 60)
    df['below_30'] = (df.elev <= - 79)
    df_sum = df.groupby('cc').sum()
    # find connected components that are above ground and connect deeper than 30m below
    df_conn = df_sum[(df_sum.above_gse>0).values & (df_sum.below_30 >0).values].index
    # check if top and bottom connected is in cco array
    cco_vert = np.isin(cco, df_conn)
    return(cco_vert)

In [ ]:
cco_vert = get_cco_vert(cco, dem_data)
# tprogs_cleaning.get_tprogs_for_elev(dem_data)
tprogs_lay = tc.elev_to_tprogs_layers(elev=dem_data, tprogs_info=tprogs_info)
# elev_to_tprogs_layers?
rows = np.where(np.ones(tprogs_lay.shape)==1)[0]
cols = np.where(np.ones(tprogs_lay.shape)==1)[1]

In [ ]:
# if comparing against cco with above land cropped
conn_arr[dem_data<bot_elev]=0

In [ ]:
conn_arr.shape

In [ ]:
# this 1% doesn't change between setting bottom 30 m below to the model bottom
# 1% of CCOs are not vertically connected
(cco_vert>0).sum()/(320*100*230), conn_arr.sum()/(320*100*230)

In [ ]:
# cropped above land surface
(cco_vert>0).sum()/(bot_elev<dem_data).sum(), conn_arr.sum()/(bot_elev<dem_data).sum()

In [ ]:
cco_plt = np.copy(cco_vert).astype(int)
cco_plt[cco_vert != conn_arr] = 2

# plt.imshow(q_plt)
plt.imshow(cco_plt[150])
plt.colorbar(shrink=0.6)
plt.show()


In [ ]:

def conn_count(cpts, str_setbacks):
    tic = time.time()
#     setbacks = np.unique(str_setbacks)
    setbacks = np.arange(0, str_setbacks.shape[0])
    setbacks = setbacks[~np.isnan(setbacks)]*200
    # will count total number of cells for each setback distance and for all 100 realizations
    hf_tot = np.zeros((100,len(setbacks)))
    # layer for each local setback
    num_sites = len(np.unique(local_str_setbacks))-1
    hf_tot_local = np.zeros((num_sites, 100,len(setbacks)))
    hf_all = np.zeros((100, 100, 230)) # map high flow for each realization
    
    # dataframe for grouping and area analysis
    stat_cols = ['Num_Grps','Mean','Median','Min','Max','Variance','Realization', 'Setback']
    cell_stats_all = pd.DataFrame(np.zeros((100*len(str_setbacks),len(stat_cols))), columns=stat_cols)
    local_cols = stat_cols+['Location']
    cell_stats_all_local = pd.DataFrame(np.zeros((num_sites*100*len(str_setbacks),len(local_cols))), columns=local_cols)
    
    k=0 # counter 
    kl = 0 # local counter
    for r in np.arange(0,100):
        print('Realization', r, ' time since start %.2f min' %((time.time()-tic)/60), end='. ')
        cco_in = np.loadtxt(join(model_ws, 'r'+str(r).zfill(3)+'.CCO'))
        # reshape to array format
#         cco = np.reshape(cco_in, arr_dim)
        cco = np.reshape(cco_in, np.flip(arr_dim))
        cco = np.transpose(cco)
        cco_vert = get_cco_vert(cco, dem_data)
        print('Percent connected %.2f '%((cco_vert>0).sum()*100/(320*100*230)))
        # get high conductivity at ground surface
        q_lay = np.zeros((100,230))
        q_lay[rows,cols] = cco_vert[tprogs_lay[rows,cols],rows,cols] 
        
        hf_all[r,:] = np.copy(q_lay)
        # complete analysis for regional and local setbacks
        for n in np.arange(0,len(setbacks)):
            # overlay high flow cells with setback distance
#             q_lay_setback = tp.overlay_hf_setback(q_lay, str_setbacks < n+1)
            q_lay_setback = tp.overlay_hf_setback(q_lay, str_setbacks[n,:,:])
            # calculate total cells in each setback
            hf_tot[r,n] = q_lay_setback.sum()
            # calculate high flow groups and summary statistics
            cell_stats_all.iloc[k] = tp.calc_area_stats(r,n, 0, q_lay_setback, stat_cols)
            k +=1
            # iterate over local setbacks
            for l in np.unique(local_str_setbacks)[1:].astype(int):
                arr = np.zeros(local_str_setbacks[n,:,:].shape)
                arr[local_str_setbacks[n,::]==l] = 1
                q_lay_setback_local = tp.overlay_hf_setback(q_lay, arr)
#                 q_lay_setback_local = overlay_hf_setback(q_lay, arr)
                hf_tot_local[l-1,r,n] = q_lay_setback_local.sum()
                cell_stats_all_local.iloc[kl] = tp.calc_area_stats(r,n,l, q_lay_setback_local, local_cols)
                kl+=1
    hf_tot_df = pd.DataFrame(hf_tot, columns = setbacks)
    hf_tot_local = np.reshape(hf_tot_local, (num_sites*100,len(setbacks)))
    hf_tot_local_df = pd.DataFrame(hf_tot_local, columns = setbacks)
    hf_all_out = np.reshape(hf_all, (100*100, 230))
    np.savetxt(data_dir+'surface_highflow_by_realization_'+str(cpts)+'.tsv', hf_all_out, delimiter = '\t')
    
    # save counted high flow cells to a csv
    hf_tot_df.to_csv(data_dir+'surface_highflow_by_distance_regional_'+str(cpts)+'.csv', index=False)
    hf_tot_local_df.to_csv(data_dir+'surface_highflow_by_distance_local_'+str(cpts)+'.csv', index=False)
    # save grouping analysis and area statistics
    cell_stats_all.to_csv(data_dir+'surface_highflow_cells_statistics_regional'+str(cpts)+'.csv', index=False)
    cell_stats_all_local.to_csv(data_dir+'surface_highflow_cells_statistics_local'+str(cpts)+'.csv', index=False)

    toc = time.time()
    print('Total time was', (toc-tic)/60, 'minutes.')

The range of connected facies meeting vertical criteria is 16.5-17.5 out of 17.8 so not huge variability but not enough to be interesting

In [ ]:
# the two models that ran independently with new dat files was r000 and r009 which show a coarse of 16%, issue with input?
# it's because those realizations cropped the data above land surface so they have a different default
conn_count(6, str_setbacks)
